In [1]:
import logging
import pandas as pd
from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from datetime import datetime
from bs4 import BeautifulSoup
from datetime import datetime
import requests




# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# HighBond API configuration
API_TOKEN = "e6589a07a91e1604d711c78ef1b8c091fc7c09119f11d2c4aad948fc53942675"
BASE_URL = "https://apis-eu.highbond.com/v1/orgs/48414"
HEADERS = {
    "Authorization": f"Bearer {API_TOKEN}",
    "Content-Type": "application/vnd.api+json"
}

def get_all_projects():
    url = f"{BASE_URL}/projects"
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        return response.json()['data']
    else:
        logger.error(f"Failed to get projects: {response.status_code} - {response.text}")
        return []

def get_project_issues(project_id):
    url = f"{BASE_URL}/projects/{project_id}/issues"
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        return response.json()['data']
    else:
        logger.error(f"Failed to get issues for project {project_id}: {response.status_code} - {response.text}")
        return []

def clean_html(value):
    if isinstance(value, (int, float)):
        return str(value)
    if not isinstance(value, str):
        return str(value)

    soup = BeautifulSoup(value, 'html.parser')

    # Check if there is a <table> tag and if so, convert it to a Word table
    if soup.find('table'):
        return convert_html_table_to_word(soup.find('table'))
    
    for script in soup(["script", "style"]):
        script.decompose()
    for br in soup.find_all("br"):
        br.replace_with("\n")
    text = soup.get_text()
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return text

def convert_html_table_to_word(table_soup):
    """Converts an HTML table into a Word table."""
    table = []
    rows = table_soup.find_all('tr')

    for row in rows:
        cells = row.find_all(['td', 'th'])
        table.append([cell.get_text(strip=True) for cell in cells])
    
    word_table = ""
    for row in table:
        word_table += "| " + " | ".join(row) + " |\n"

    return word_table

def prompt_filters():
    region_filter = input("Enter region filter (partial match allowed): ")
    month_filter = input("Enter month filter (YYYY-MM): ")
    return region_filter, month_filter

def create_word_report(table_data, headers, region_filter, month_filter):
    doc = Document()

    # -------- Set up styles --------
    normal_style = doc.styles['Normal']
    normal_font = normal_style.font
    normal_font.name = 'Calibri'
    normal_font.size = Pt(11)

    h1_style = doc.styles['Heading 1']
    h1_font = h1_style.font
    h1_font.name = 'Calibri'
    h1_font.size = Pt(16)
    h1_font.color.rgb = RGBColor.from_string('107AB8')

    h2_style = doc.styles['Heading 2']
    h2_font = h2_style.font
    h2_font.name = 'Calibri'
    h2_font.size = Pt(12)
    h2_font.color.rgb = RGBColor.from_string('EF6149')

    # -------- Header with logos --------
    header = doc.sections[0].header
    header_table = header.add_table(rows=1, cols=2)
    header_table.autofit = True
    cells = header_table.rows[0].cells

    try:
        cells[0].paragraphs[0].add_run().add_picture('minigroup_logo.png', width=Inches(1.5))
        right_para = cells[1].paragraphs[0]
        right_para.alignment = WD_ALIGN_PARAGRAPH.RIGHT
        right_para.add_run().add_picture('eleven_degrees_logo.png', width=Inches(1.5))
    except Exception as e:
        logger.warning(f"Logo image(s) not found or failed to load: {e}")

    # -------- Cover Page --------
    title = doc.add_paragraph()
    run = title.add_run("Regional Issues Report")
    run.font.name = 'Calibri'
    run.font.size = Pt(24)
    run.font.bold = True
    run.font.color.rgb = RGBColor.from_string('107AB8')
    title.alignment = WD_ALIGN_PARAGRAPH.CENTER

    sub = doc.add_paragraph()
    sub_run = sub.add_run("Mini Group / Eleven Degrees Consulting")
    sub_run.font.name = 'Calibri'
    sub_run.font.size = Pt(14)
    sub_run.font.color.rgb = RGBColor(0, 0, 0)
    sub.alignment = WD_ALIGN_PARAGRAPH.CENTER

    date = doc.add_paragraph()
    date_run = date.add_run(f"Date: {datetime.today().strftime('%Y-%m-%d')}")
    date_run.font.name = 'Calibri'
    date_run.font.size = Pt(12)
    date_run.font.color.rgb = RGBColor(0, 0, 0)
    date.alignment = WD_ALIGN_PARAGRAPH.CENTER

    doc.add_page_break()

    # -------- Table of Contents --------
    toc_paragraph = doc.add_paragraph()
    run = toc_paragraph.add_run()
    fldChar = OxmlElement('w:fldChar')
    fldChar.set(qn('w:fldCharType'), 'begin')
    instrText = OxmlElement('w:instrText')
    instrText.set(qn('xml:space'), 'preserve')
    instrText.text = 'TOC \\o "1-3" \\h \\z \\u'
    fldChar2 = OxmlElement('w:fldChar')
    fldChar2.set(qn('w:fldCharType'), 'separate')
    t = OxmlElement('w:t')
    t.text = "Right-click to update Table of Contents"
    fldChar2.append(t)
    fldChar3 = OxmlElement('w:fldChar')
    fldChar3.set(qn('w:fldCharType'), 'end')
    r_element = run._r
    r_element.append(fldChar)
    r_element.append(instrText)
    r_element.append(fldChar2)
    r_element.append(fldChar3)

    doc.add_page_break()

    # -------- Project & Issues --------
    current_project = None

    for row in table_data:
        project_id, project_name = row[0], row[1]

        if current_project != project_id:
            doc.add_heading(f"Project: {project_name}", level=1)
            doc.add_paragraph(f"Branch: {row[2]}")
            doc.add_paragraph(f"Region: {row[3]}")
            doc.add_paragraph(f"Start Date: {row[4]}")
            doc.add_paragraph(f"Status: {row[5]}")
            doc.add_paragraph(f"Branch Manager: {row[14]}")
            doc.add_paragraph(f"Operations Manager: {row[15]}")
            doc.add_paragraph(f"Supervisor: {row[16]}")
            current_project = project_id

        doc.add_heading(f"Issue: {row[6]}", level=2)
        table = doc.add_table(rows=0, cols=2)
        table.style = 'Light List Accent 1'

        def add_row(label, value):
            row_cells = table.add_row().cells
            row_cells[0].text = label
            row_cells[1].text = clean_html(value)

        add_row("Severity", row[7])
        add_row("Description", row[8])
        add_row("Implication", row[9])
        add_row("Cost Impact", f"${row[10]}")
        add_row("Management Comment 1", row[11])
        add_row("Management Comment 2", row[12])
        add_row("Recommendation", row[13])
        doc.add_paragraph()

    return doc


def main():
    try:
        logger.info("Starting the script")
        region_filter, month_filter = prompt_filters()
        logger.info(f"Filters: Region - {region_filter}, Month - {month_filter}")

        projects = get_all_projects()
        logger.info(f"Retrieved {len(projects)} projects")

        table_data = []
        headers = [
            "Project ID", "Project Name", "Branch", "Region", "Start Date", "Status",
            "Issue Title", "Severity", "Description", "Implication", "Cost Impact",
            "Management Comments 1", "Management Comments 2", "Recommendation",
            "Branch Manager", "Operations Manager", "Supervisor"
        ]

        for project in projects:
            try:
                project_id = project.get("id")
                if not project_id:
                    logger.warning(f"Project ID not found: {project}")
                    continue

                project_attributes = project.get("attributes", {})
                project_name = project_attributes.get("name", "N/A")
                start_date = project_attributes.get("start_date", "")
                status = project_attributes.get("status", "N/A")
                project_custom_fields = project_attributes.get("custom_attributes", [])

                region = next((f.get("value", "N/A") for f in project_custom_fields if f.get("term") == "Custom field 2"), "N/A")
                branch = next((f.get("value", "N/A") for f in project_custom_fields if f.get("term") == "Custom field 8"), "N/A")
                branch_manager = next((f.get("value", "N/A") for f in project_custom_fields if f.get("term") == "Custom field 5"), "N/A")
                operations_manager = next((f.get("value", "N/A") for f in project_custom_fields if f.get("term") == "Custom field 6"), "N/A")
                supervisor = next((f.get("value", "N/A") for f in project_custom_fields if f.get("term") == "Custom field 7"), "N/A")

                if region_filter and region_filter.lower() not in region.lower():
                    logger.info(f"Skipping project {project_name} due to region filter")
                    continue
                if month_filter and not start_date.startswith(month_filter):
                    logger.info(f"Skipping project {project_name} due to month filter")
                    continue

                issues = get_project_issues(project_id)
                if len(issues) == 0:
                    continue

                for issue in issues:
                    try:
                        attributes = issue.get("attributes", {})
                        title = attributes.get("title", "")
                        severity = attributes.get("severity", "")
                        description = clean_html(attributes.get("description", ""))
                        implication = clean_html(attributes.get("effect", ""))
                        cost_impact = str(attributes.get("cost_impact", 0))
                        recommendation = clean_html(attributes.get("recommendation", ""))

                        issue_custom_attributes = attributes.get("custom_attributes", [])
                        mgmt_comment_1 = next((field.get("value", "") for field in issue_custom_attributes if field.get("term") == "Custom field 1"), "")
                        mgmt_comment_2 = next((field.get("value", "") for field in issue_custom_attributes if field.get("term") == "Custom field 2"), "")

                        row = [
                            project_id, project_name, branch, region, start_date, status,
                            title, severity, description, implication, cost_impact,
                            mgmt_comment_1, mgmt_comment_2, recommendation,
                            branch_manager, operations_manager, supervisor
                        ]
                        table_data.append(row)
                    except Exception as e:
                        logger.error(f"Error processing issue for project {project_name}: {str(e)}")

            except Exception as e:
                logger.error(f"Error processing project {project.get('id', 'Unknown')}: {str(e)}")

        logger.info(f"Total issue rows prepared: {len(table_data)}")

        if len(table_data) == 0:
            logger.warning("No matching issues found")
        else:
            df = pd.DataFrame(table_data, columns=headers)
            df.to_csv("project_data.csv", index=False)
            logger.info("Data saved to 'project_data.csv'")

            doc = create_word_report(table_data, headers, region_filter, month_filter)
            doc.save("project_report.docx")
            logger.info("Report generated and saved as 'project_report.docx'")

    except Exception as e:
        logger.error(f"An error occurred in the main function: {str(e)}")
        logger.exception("Exception details:")

if __name__ == "__main__":
    main()

2025-06-16 11:36:35,879 - INFO - Starting the script
2025-06-16 11:36:48,781 - INFO - Filters: Region - , Month - 2025-2
2025-06-16 11:36:50,340 - INFO - Retrieved 50 projects
2025-06-16 11:36:50,341 - INFO - Skipping project Akiyda 2000 Ltd due to month filter
2025-06-16 11:36:50,341 - INFO - Skipping project Findings Follow-Up Tracker due to month filter
2025-06-16 11:36:50,342 - INFO - Skipping project Thika due to month filter
2025-06-16 11:36:50,342 - INFO - Skipping project Nakuru Branch due to month filter
2025-06-16 11:36:50,343 - INFO - Skipping project Crater Branch due to month filter
2025-06-16 11:36:50,345 - INFO - Skipping project Kitui Branch due to month filter
2025-06-16 11:36:50,346 - INFO - Skipping project Meru due to month filter
2025-06-16 11:36:50,347 - INFO - Skipping project Machakos Branch due to month filter
2025-06-16 11:36:50,347 - INFO - Skipping project Nyamasaria Branch due to month filter
2025-06-16 11:36:50,348 - INFO - Skipping project Majengo 1 due t

In [2]:
import sys
!{sys.executable} -m pip install python-docx


In [3]:
import logging
import pandas as pd

from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.section import WD_ORIENT
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from datetime import datetime
from bs4 import BeautifulSoup
import requests

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# HighBond API configuration
API_TOKEN = "e6589a07a91e1604d711c78ef1b8c091fc7c09119f11d2c4aad948fc53942675"
BASE_URL = "https://apis-eu.highbond.com/v1/orgs/48414"
HEADERS = {
    "Authorization": f"Bearer {API_TOKEN}",
    "Content-Type": "application/vnd.api+json"
}

def get_all_projects():
    url = f"{BASE_URL}/projects"
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        return response.json()['data']
    else:
        logger.error(f"Failed to get projects: {response.status_code} - {response.text}")
        return []

def get_project_issues(project_id):
    url = f"{BASE_URL}/projects/{project_id}/issues"
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        return response.json()['data']
    else:
        logger.error(f"Failed to get issues for project {project_id}: {response.status_code} - {response.text}")
        return []

def clean_html(value):
    if isinstance(value, (int, float)):
        return str(value)
    if not isinstance(value, str):
        return str(value)
    soup = BeautifulSoup(value, 'html.parser')
    if soup.find('table'):
        return convert_html_table_to_word(soup.find('table'))
    for tag in soup(["script", "style"]): tag.decompose()
    for br in soup.find_all("br"): br.replace_with("\n")
    text = soup.get_text()
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    return '\n'.join(chunk for chunk in chunks if chunk)

def convert_html_table_to_word(table_soup):
    table = []
    for row in table_soup.find_all('tr'):
        cells = row.find_all(['td', 'th'])
        table.append([cell.get_text(strip=True) for cell in cells])
    word_table = ""
    for row in table:
        word_table += "| " + " | ".join(row) + " |\n"
    return word_table

def prompt_filters():
    region_filter = input("Enter region filter (partial match allowed): ")
    month_filter = input("Enter month filter (YYYY-MM): ")
    return region_filter, month_filter

def add_background_footer(section):
    footer = section.footer
    paragraph = footer.paragraphs[0]
    run = paragraph.add_run()
    run.text = " " * 200  # force footer area height
    shading_elm = OxmlElement('w:shd')
    shading_elm.set(qn('w:fill'), '107AB8')  # background color
    shading_elm.set(qn('w:val'), 'clear')
    run._r.get_or_add_rPr().append(shading_elm)

def create_word_report(table_data, headers, region_filter, month_filter):
    doc = Document()

    # Set landscape orientation and margins
    section = doc.sections[0]
    section.orientation = WD_ORIENT.LANDSCAPE
    section.page_width, section.page_height = Inches(11), Inches(8.5)
    for s in doc.sections:
        s.top_margin = s.bottom_margin = Inches(0.5)
        s.left_margin = s.right_margin = Inches(0.5)

    # Define styles
    styles = doc.styles
    styles['Normal'].font.name = 'Calibri'
    styles['Normal'].font.size = Pt(11)

    styles['Heading 1'].font.size = Pt(16)
    styles['Heading 1'].font.color.rgb = RGBColor.from_string('107AB8')

    styles['Heading 2'].font.size = Pt(13)
    styles['Heading 2'].font.color.rgb = RGBColor.from_string('EF6149')

    # Cover Page
    doc.add_paragraph().add_run("Regional Issues Report").bold = True
    doc.paragraphs[-1].alignment = WD_ALIGN_PARAGRAPH.CENTER
    doc.paragraphs[-1].runs[0].font.size = Pt(24)
    doc.paragraphs[-1].runs[0].font.color.rgb = RGBColor.from_string('107AB8')

    subtitle = doc.add_paragraph("Mini Group / Eleven Degrees Consulting")
    subtitle.alignment = WD_ALIGN_PARAGRAPH.CENTER
    subtitle.runs[0].font.size = Pt(14)

    doc.add_paragraph(f"Date: {datetime.today().strftime('%Y-%m-%d')}").alignment = WD_ALIGN_PARAGRAPH.CENTER
    add_background_footer(doc.sections[0])
    doc.add_page_break()

    # Table of Contents
    toc = doc.add_paragraph()
    run = toc.add_run()
    fldChar = OxmlElement('w:fldChar'); fldChar.set(qn('w:fldCharType'), 'begin')
    instrText = OxmlElement('w:instrText'); instrText.text = 'TOC \\o "1-2" \\h \\z \\u'
    fldChar2 = OxmlElement('w:fldChar'); fldChar2.set(qn('w:fldCharType'), 'separate')
    fldChar3 = OxmlElement('w:fldChar'); fldChar3.set(qn('w:fldCharType'), 'end')
    r_element = run._r
    r_element.append(fldChar)
    r_element.append(instrText)
    r_element.append(fldChar2)
    r_element.append(fldChar3)
    doc.add_page_break()

    # Group data by project
    from collections import defaultdict
    grouped_data = defaultdict(list)
    for row in table_data:
        if row[7].lower() in ['high', 'medium']:
            grouped_data[row[0]].append(row)

    for pid, rows in grouped_data.items():
        project_name = rows[0][1]
        branch, region, start_date, status = rows[0][2], rows[0][3], rows[0][4], rows[0][5]
        bm, om, sup = rows[0][14], rows[0][15], rows[0][16]

        # -- Page Header per project --
        section = doc.add_section(start_type=1)
        header = section.header
        header_table = header.add_table(rows=2, cols=3, width=Inches(9))
        header_table.autofit = True
        header_table.alignment = WD_ALIGN_PARAGRAPH.CENTER

        try:
            header_table.cell(0, 0).paragraphs[0].add_run().add_picture("minigroup_logo.png", width=Inches(1.2))
            header_table.cell(0, 2).paragraphs[0].add_run().add_picture("eleven_degrees.jpg", width=Inches(1.2))
        except Exception as e:
            logger.warning(f"Header logo loading failed: {e}")

        header_text = header_table.cell(1, 1).paragraphs[0]
        header_text.alignment = WD_ALIGN_PARAGRAPH.CENTER
        run = header_text.add_run(f"Branch: {branch} | Region: {region} | Start: {start_date} | Status: {status}\n"
                                  f"BM: {bm} | OM: {om} | Sup: {sup}")
        run.font.size = Pt(9)

        doc.add_heading(f"Project: {project_name}", level=1)

        for row in rows:
            doc.add_heading(f"Issue: {row[6]}", level=2)
            table = doc.add_table(rows=0, cols=2)
            table.style = 'Light List Accent 1'

            def add_row(label, value):
                row_cells = table.add_row().cells
                row_cells[0].text = label
                row_cells[1].text = clean_html(value)

            add_row("Severity", row[7])
            add_row("Description", row[8])
            add_row("Implication", row[9])
            add_row("Cost Impact", f"${row[10]}")
            add_row("Management Comment 1", row[11])
            add_row("Management Comment 2", row[12])
            add_row("Recommendation", row[13])
            doc.add_paragraph()

    # Custom Footer with Styled Page Numbers
    for section in doc.sections:
        footer = section.footer
        para = footer.paragraphs[0]
        para.alignment = WD_ALIGN_PARAGRAPH.RIGHT
        run = para.add_run()
        fldChar = OxmlElement('w:fldChar'); fldChar.set(qn('w:fldCharType'), 'begin')
        instrText = OxmlElement('w:instrText'); instrText.text = 'PAGE'
        fldChar2 = OxmlElement('w:fldChar'); fldChar2.set(qn('w:fldCharType'), 'separate')
        fldChar3 = OxmlElement('w:fldChar'); fldChar3.set(qn('w:fldCharType'), 'end')
        r_element = run._r
        r_element.append(fldChar)
        r_element.append(instrText)
        r_element.append(fldChar2)
        r_element.append(fldChar3)
        run.font.size = Pt(9)
        run.font.color.rgb = RGBColor.from_string('808080')

    return doc

def main():
    try:
        logger.info("Starting script")
        region_filter, month_filter = prompt_filters()
        projects = get_all_projects()
        table_data = []
        headers = [
            "Project ID", "Project Name", "Branch", "Region", "Start Date", "Status",
            "Issue Title", "Severity", "Description", "Implication", "Cost Impact",
            "Management Comments 1", "Management Comments 2", "Recommendation",
            "Branch Manager", "Operations Manager", "Supervisor"
        ]

        for project in projects:
            try:
                pid = project.get("id")
                if not pid:
                    continue
                attr = project.get("attributes", {})
                name = attr.get("name", "N/A")
                start = attr.get("start_date", "")
                status = attr.get("status", "N/A")
                custom = attr.get("custom_attributes", [])

                region = next((f.get("value", "N/A") for f in custom if f.get("term") == "Custom field 2"), "N/A")
                branch = next((f.get("value", "N/A") for f in custom if f.get("term") == "Custom field 8"), "N/A")
                bm = next((f.get("value", "N/A") for f in custom if f.get("term") == "Custom field 5"), "N/A")
                om = next((f.get("value", "N/A") for f in custom if f.get("term") == "Custom field 6"), "N/A")
                sup = next((f.get("value", "N/A") for f in custom if f.get("term") == "Custom field 7"), "N/A")

                if region_filter and region_filter.lower() not in region.lower(): continue
                if month_filter and not start.startswith(month_filter): continue

                issues = get_project_issues(pid)
                if not issues: continue

                for issue in issues:
                    severity = issue["attributes"].get("severity", "").lower()
                    if severity not in ["high", "medium"]:
                        continue
                    i_attr = issue.get("attributes", {})
                    title = i_attr.get("title", "")
                    desc = clean_html(i_attr.get("description", ""))
                    effect = clean_html(i_attr.get("effect", ""))
                    cost = str(i_attr.get("cost_impact", 0))
                    rec = clean_html(i_attr.get("recommendation", ""))
                    icustom = i_attr.get("custom_attributes", [])
                    cm1 = next((f.get("value", "") for f in icustom if f.get("term") == "Custom field 1"), "")
                    cm2 = next((f.get("value", "") for f in icustom if f.get("term") == "Custom field 2"), "")

                    row = [pid, name, branch, region, start, status, title, severity, desc, effect, cost, cm1, cm2, rec, bm, om, sup]
                    table_data.append(row)

            except Exception as e:
                logger.error(f"Error processing project {project.get('id')}: {e}")

        if table_data:
            df = pd.DataFrame(table_data, columns=headers)
            df.to_csv("project_data.csv", index=False)
            logger.info("CSV saved")

            doc = create_word_report(table_data, headers, region_filter, month_filter)
            doc.save("project_report.docx")
            logger.info("Word report saved")

        else:
            logger.warning("No matching issues found")

    except Exception as e:
        logger.error(f"Main error: {e}")
        logger.exception("Traceback:")

if __name__ == "__main__":
    main()


2025-06-16 11:36:52,578 - INFO - Starting script
2025-06-16 11:37:21,917 - WARNING - No matching issues found


In [4]:
import logging
import pandas as pd
from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.section import WD_ORIENT
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from datetime import datetime
from bs4 import BeautifulSoup
import requests

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# HighBond API configuration
API_TOKEN = "e6589a07a91e1604d711c78ef1b8c091fc7c09119f11d2c4aad948fc53942675"
BASE_URL = "https://apis-eu.highbond.com/v1/orgs/48414"
HEADERS = {
    "Authorization": f"Bearer {API_TOKEN}",
    "Content-Type": "application/vnd.api+json"
}

def get_all_projects():
    url = f"{BASE_URL}/projects"
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        return response.json()['data']
    else:
        logger.error(f"Failed to get projects: {response.status_code} - {response.text}")
        return []

def get_project_issues(project_id):
    url = f"{BASE_URL}/projects/{project_id}/issues"
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        return response.json()['data']
    else:
        logger.error(f"Failed to get issues for project {project_id}: {response.status_code} - {response.text}")
        return []

def clean_html(value):
    if isinstance(value, (int, float)):
        return str(value)
    if not isinstance(value, str):
        return str(value)
    soup = BeautifulSoup(value, 'html.parser')
    if soup.find('table'):
        return convert_html_table_to_word(soup.find('table'))
    for tag in soup(["script", "style"]): tag.decompose()
    for br in soup.find_all("br"): br.replace_with("\n")
    text = soup.get_text()
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    return '\n'.join(chunk for chunk in chunks if chunk)

def convert_html_table_to_word(table_soup):
    table = []
    for row in table_soup.find_all('tr'):
        cells = row.find_all(['td', 'th'])
        table.append([cell.get_text(strip=True) for cell in cells])
    word_table = ""
    for row in table:
        word_table += "| " + " | ".join(row) + " |\n"
    return word_table

def prompt_filters():
    region_filter = input("Enter region filter (partial match allowed): ")
    month_filter = input("Enter month filter (YYYY-MM): ")
    return region_filter, month_filter

def add_background_footer(section):
    footer = section.footer
    paragraph = footer.paragraphs[0]
    run = paragraph.add_run(" " * 200)  # Maintain footer height
    shading_elm = OxmlElement('w:shd')
    shading_elm.set(qn('w:fill'), '107AB8')
    shading_elm.set(qn('w:val'), 'clear')
    run._r.get_or_add_rPr().append(shading_elm)

def add_page_number(section):
    footer = section.footer
    para = footer.paragraphs[0]
    para.clear()
    para.alignment = WD_ALIGN_PARAGRAPH.RIGHT
    run = para.add_run()
    fldChar1 = OxmlElement('w:fldChar')
    fldChar1.set(qn('w:fldCharType'), 'begin')
    instrText = OxmlElement('w:instrText')
    instrText.text = 'PAGE'
    fldChar2 = OxmlElement('w:fldChar')
    fldChar2.set(qn('w:fldCharType'), 'separate')
    fldChar3 = OxmlElement('w:fldChar')
    fldChar3.set(qn('w:fldCharType'), 'end')
    run._r.append(fldChar1)
    run._r.append(instrText)
    run._r.append(fldChar2)
    run._r.append(fldChar3)
    run.font.size = Pt(9)
    run.font.color.rgb = RGBColor.from_string('808080')

def create_word_report(table_data, headers, region_filter, month_filter):
    doc = Document()

    # Page settings
    section = doc.sections[0]
    section.orientation = WD_ORIENT.LANDSCAPE
    section.page_width, section.page_height = Inches(11), Inches(8.5)
    for s in doc.sections:
        s.top_margin = s.bottom_margin = Inches(0.5)
        s.left_margin = s.right_margin = Inches(0.5)

    # Fonts and styles
    styles = doc.styles
    styles['Normal'].font.name = 'Calibri'
    styles['Normal'].font.size = Pt(11)
    styles['Heading 1'].font.size = Pt(16)
    styles['Heading 1'].font.color.rgb = RGBColor.from_string('107AB8')
    styles['Heading 2'].font.size = Pt(13)
    styles['Heading 2'].font.color.rgb = RGBColor.from_string('EF6149')

    # Cover Page (no header/footer)
    doc.add_paragraph().add_run("Regional Issues Report").bold = True
    doc.paragraphs[-1].alignment = WD_ALIGN_PARAGRAPH.CENTER
    doc.paragraphs[-1].runs[0].font.size = Pt(24)
    doc.paragraphs[-1].runs[0].font.color.rgb = RGBColor.from_string('107AB8')

    subtitle = doc.add_paragraph("Mini Group / Eleven Degrees Consulting")
    subtitle.alignment = WD_ALIGN_PARAGRAPH.CENTER
    subtitle.runs[0].font.size = Pt(14)

    doc.add_paragraph(f"Date: {datetime.today().strftime('%Y-%m-%d')}").alignment = WD_ALIGN_PARAGRAPH.CENTER
    doc.add_page_break()

    # TOC
    toc = doc.add_paragraph()
    run = toc.add_run()
    fldChar = OxmlElement('w:fldChar'); fldChar.set(qn('w:fldCharType'), 'begin')
    instrText = OxmlElement('w:instrText'); instrText.text = 'TOC \\o "1-2" \\h \\z \\u'
    fldChar2 = OxmlElement('w:fldChar'); fldChar2.set(qn('w:fldCharType'), 'separate')
    fldChar3 = OxmlElement('w:fldChar'); fldChar3.set(qn('w:fldCharType'), 'end')
    r_element = run._r
    r_element.append(fldChar)
    r_element.append(instrText)
    r_element.append(fldChar2)
    r_element.append(fldChar3)
    doc.add_page_break()

    # Group and write project data
    from collections import defaultdict
    grouped_data = defaultdict(list)
    for row in table_data:
        if str(row[7]).lower() in ['high', 'medium']:
            grouped_data[row[0]].append(row)

    for pid, rows in grouped_data.items():
        project_name = rows[0][1]
        branch, region, start_date, status = rows[0][2], rows[0][3], rows[0][4], rows[0][5]
        bm, om, sup = rows[0][14], rows[0][15], rows[0][16]

        section = doc.add_section(start_type=1)
        section.orientation = WD_ORIENT.LANDSCAPE
        section.page_width, section.page_height = Inches(11), Inches(8.5)

        # Header with logos and project info
        header = section.header
        header_table = header.add_table(rows=2, cols=3)
        header_table.autofit = True

        try:
            header_table.cell(0, 0).paragraphs[0].add_run().add_picture("minigroup_logo.png", width=Inches(1.2))
            header_table.cell(0, 2).paragraphs[0].add_run().add_picture("eleven_degrees.jpg", width=Inches(1.2))
        except Exception as e:
            logger.warning(f"Header logo loading failed: {e}")

        header_text = header_table.cell(1, 1).paragraphs[0]
        header_text.alignment = WD_ALIGN_PARAGRAPH.CENTER
        run = header_text.add_run(
            f"Branch: {branch} | Region: {region} | Start: {start_date} | Status: {status}\n"
            f"BM: {bm} | OM: {om} | Sup: {sup}")
        run.font.size = Pt(9)

        doc.add_heading(f"Project: {project_name}", level=1)

        for row in rows:
            doc.add_heading(f"Issue: {row[6]}", level=2)
            table = doc.add_table(rows=0, cols=2)
            table.style = 'Light List Accent 1'

            def add_row(label, value):
                row_cells = table.add_row().cells
                row_cells[0].text = label
                row_cells[1].text = clean_html(value)

            add_row("Severity", row[7])
            add_row("Description", row[8])
            add_row("Implication", row[9])
            add_row("Cost Impact", f"${row[10]}")
            add_row("Management Comment 1", row[11])
            add_row("Management Comment 2", row[12])
            add_row("Recommendation", row[13])
            doc.add_paragraph()

        # Add page number to footer
        add_page_number(section)

    return doc

def main():
    try:
        logger.info("Starting script")
        region_filter, month_filter = prompt_filters()
        projects = get_all_projects()
        table_data = []
        headers = [
            "Project ID", "Project Name", "Branch", "Region", "Start Date", "Status",
            "Issue Title", "Severity", "Description", "Implication", "Cost Impact",
            "Management Comments 1", "Management Comments 2", "Recommendation",
            "Branch Manager", "Operations Manager", "Supervisor"
        ]

        for project in projects:
            try:
                pid = project.get("id")
                if not pid:
                    continue
                attr = project.get("attributes", {})
                name = attr.get("name", "N/A")
                start = attr.get("start_date", "")
                status = attr.get("status", "N/A")
                custom = attr.get("custom_attributes", [])

                region = next((f.get("value", "N/A") for f in custom if f.get("term") == "Custom field 2"), "N/A")
                branch = next((f.get("value", "N/A") for f in custom if f.get("term") == "Custom field 8"), "N/A")
                bm = next((f.get("value", "N/A") for f in custom if f.get("term") == "Custom field 5"), "N/A")
                om = next((f.get("value", "N/A") for f in custom if f.get("term") == "Custom field 6"), "N/A")
                sup = next((f.get("value", "N/A") for f in custom if f.get("term") == "Custom field 7"), "N/A")

                if region_filter and region_filter.lower() not in str(region).lower():
                    continue
                if month_filter and not str(start).startswith(month_filter):
                    continue

                issues = get_project_issues(pid)
                if not issues:
                    continue

                for issue in issues:
                    severity = str(issue["attributes"].get("severity", "")).lower()
                    if severity not in ["high", "medium"]:
                        continue
                    i_attr = issue.get("attributes", {})
                    title = i_attr.get("title", "")
                    desc = clean_html(i_attr.get("description", ""))
                    effect = clean_html(i_attr.get("effect", ""))
                    cost = str(i_attr.get("cost_impact", 0))
                    rec = clean_html(i_attr.get("recommendation", ""))
                    icustom = i_attr.get("custom_attributes", [])
                    cm1 = next((f.get("value", "") for f in icustom if f.get("term") == "Custom field 1"), "")
                    cm2 = next((f.get("value", "") for f in icustom if f.get("term") == "Custom field 2"), "")

                    row = [pid, name, branch, region, start, status, title, severity, desc, effect, cost, cm1, cm2, rec, bm, om, sup]
                    table_data.append(row)

            except Exception as e:
                logger.error(f"Error processing project {project.get('id')}: {e}")

        if table_data:
            df = pd.DataFrame(table_data, columns=headers)
            df.to_csv("project_data.csv", index=False)
            logger.info("CSV saved")

            doc = create_word_report(table_data, headers, region_filter, month_filter)
            doc.save("project_report.docx")
            logger.info("Word report saved")
        else:
            logger.warning("No matching issues found")

    except Exception as e:
        logger.error(f"Main error: {e}")
        logger.exception("Traceback:")

if __name__ == "__main__":
    main()


2025-06-16 11:37:22,042 - INFO - Starting script
2025-06-16 11:39:50,513 - INFO - CSV saved
2025-06-16 11:39:50,538 - ERROR - Main error: BlockItemContainer.add_table() missing 1 required positional argument: 'width'
2025-06-16 11:39:50,539 - ERROR - Traceback:
Traceback (most recent call last):
  File "/tmp/ipykernel_671421/1270484180.py", line 274, in main
    doc = create_word_report(table_data, headers, region_filter, month_filter)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_671421/1270484180.py", line 168, in create_word_report
    header_table = header.add_table(rows=2, cols=3)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: BlockItemContainer.add_table() missing 1 required positional argument: 'width'


In [1]:
from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.section import WD_ORIENT
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from datetime import datetime
from bs4 import BeautifulSoup
from collections import defaultdict
import pandas as pd
import logging
import requests

# Logger setup
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Constants
API_TOKEN = "acd6c44de072af279f19042267e98f0a70ca00c5966e118636dd87a451786347"
BASE_URL = "https://apis-eu.highbond.com/v1/orgs/48414"
HEADERS = {"Authorization": f"Bearer {API_TOKEN}", "Content-Type": "application/vnd.api+json"}

def get_all_projects():
    response = requests.get(f"{BASE_URL}/projects", headers=HEADERS)
    return response.json()['data'] if response.status_code == 200 else []

def get_project_issues(project_id):
    response = requests.get(f"{BASE_URL}/projects/{project_id}/issues", headers=HEADERS)
    return response.json()['data'] if response.status_code == 200 else []

def clean_html(value):
    if isinstance(value, (int, float)):
        return str(value)
    if not isinstance(value, str):
        return str(value)
    soup = BeautifulSoup(value, 'html.parser')
    for tag in soup(["script", "style"]): tag.decompose()
    for br in soup.find_all("br"): br.replace_with("\n")
    return soup.get_text().strip()

def add_page_number(section):
    footer = section.footer
    para = footer.paragraphs[0]
    para.alignment = WD_ALIGN_PARAGRAPH.RIGHT
    run = para.add_run()
    fldChar = OxmlElement('w:fldChar'); fldChar.set(qn('w:fldCharType'), 'begin')
    instrText = OxmlElement('w:instrText'); instrText.text = 'PAGE'
    fldChar2 = OxmlElement('w:fldChar'); fldChar2.set(qn('w:fldCharType'), 'separate')
    fldChar3 = OxmlElement('w:fldChar'); fldChar3.set(qn('w:fldCharType'), 'end')
    r_element = run._r
    r_element.append(fldChar)
    r_element.append(instrText)
    r_element.append(fldChar2)
    r_element.append(fldChar3)
    run.font.size = Pt(9)
    run.font.color.rgb = RGBColor.from_string('808080')

def create_word_report(table_data):
    doc = Document()
    section = doc.sections[0]
    section.orientation = WD_ORIENT.LANDSCAPE
    section.page_width, section.page_height = Inches(11), Inches(8.5)
    for s in doc.sections:
        s.top_margin = s.bottom_margin = Inches(0.5)
        s.left_margin = s.right_margin = Inches(0.5)

    styles = doc.styles
    styles['Normal'].font.name = 'Calibri'
    styles['Normal'].font.size = Pt(11)
    styles['Heading 1'].font.size = Pt(16)
    styles['Heading 1'].font.color.rgb = RGBColor.from_string('107AB8')
    styles['Heading 2'].font.size = Pt(13)
    styles['Heading 2'].font.color.rgb = RGBColor.from_string('EF6149')

    # Cover Page
    doc.add_paragraph("Regional Issues Report").alignment = WD_ALIGN_PARAGRAPH.CENTER
    doc.paragraphs[-1].runs[0].font.size = Pt(24)
    doc.paragraphs[-1].runs[0].font.color.rgb = RGBColor.from_string('107AB8')
    doc.add_paragraph("Mini Group / Eleven Degrees Consulting").alignment = WD_ALIGN_PARAGRAPH.CENTER
    doc.add_paragraph(f"Date: {datetime.today().strftime('%Y-%m-%d')}").alignment = WD_ALIGN_PARAGRAPH.CENTER
    doc.add_page_break()

    grouped_data = defaultdict(list)
    for row in table_data:
        grouped_data[row[0]].append(row)

    for pid, rows in grouped_data.items():
        section = doc.add_section(start_type=1)
        add_page_number(section)

        # Simulated header content at top of section body
        doc.add_paragraph()  # Padding space
        table = doc.add_table(rows=2, cols=3)
        table.autofit = True

        try:
            table.cell(0, 0).paragraphs[0].add_run().add_picture("minigroup_logo.png", width=Inches(1.2))
            table.cell(0, 2).paragraphs[0].add_run().add_picture("eleven_degrees.jpg", width=Inches(1.2))
        except Exception as e:
            logger.warning(f"Header logo loading failed: {e}")

        branch, region, start_date, status = rows[0][2], rows[0][3], rows[0][4], rows[0][5]
        bm, om, sup = rows[0][14], rows[0][15], rows[0][16]
        center = table.cell(1, 1).paragraphs[0]
        center.alignment = WD_ALIGN_PARAGRAPH.CENTER
        run = center.add_run(f"{branch} | {region} | {start_date} | {status}\nBM: {bm} | OM: {om} | Sup: {sup}")
        run.font.size = Pt(9)

        doc.add_paragraph()  # Spacer
        doc.add_heading(f"Project: {rows[0][1]}", level=1)

        for row in rows:
            doc.add_heading(f"Issue: {row[6]}", level=2)
            issue_table = doc.add_table(rows=0, cols=2)
            issue_table.style = 'Light List Accent 1'

            def add(label, val):
                r = issue_table.add_row().cells
                r[0].text = label
                r[1].text = clean_html(val)

            add("Severity", row[7])
            add("Description", row[8])
            add("Implication", row[9])
            add("Cost Impact", f"${row[10]}")
            add("Management Comment 1", row[11])
            add("Management Comment 2", row[12])
            add("Recommendation", row[13])
            doc.add_paragraph()

    return doc

def main():
    region_filter = input("Enter region filter (partial match allowed): ")
    month_filter = input("Enter month filter (YYYY-MM): ")

    projects = get_all_projects()
    table_data = []
    headers = [
        "Project ID", "Project Name", "Branch", "Region", "Start Date", "Status",
        "Issue Title", "Severity", "Description", "Implication", "Cost Impact",
        "Management Comments 1", "Management Comments 2", "Recommendation",
        "Branch Manager", "Operations Manager", "Supervisor"
    ]

    for project in projects:
        try:
            pid = project.get("id")
            if not pid: continue
            attr = project.get("attributes", {})
            name = attr.get("name", "N/A")
            start = attr.get("start_date", "")
            status = attr.get("status", "N/A")
            custom = attr.get("custom_attributes", [])
            region = next((f.get("value", "N/A") for f in custom if f.get("term") == "Custom field 2"), "N/A")
            branch = next((f.get("value", "N/A") for f in custom if f.get("term") == "Custom field 8"), "N/A")
            bm = next((f.get("value", "N/A") for f in custom if f.get("term") == "Custom field 5"), "N/A")
            om = next((f.get("value", "N/A") for f in custom if f.get("term") == "Custom field 6"), "N/A")
            sup = next((f.get("value", "N/A") for f in custom if f.get("term") == "Custom field 7"), "N/A")

            if region_filter and region_filter.lower() not in str(region).lower():
                continue
            if month_filter and not str(start).startswith(month_filter):
                continue

            issues = get_project_issues(pid)
            if not issues:
                continue

            for issue in issues:
                i_attr = issue.get("attributes", {})
                severity = str(i_attr.get("severity", "")).lower()
                if severity not in ["high", "medium"]:
                    continue
                title = i_attr.get("title", "")
                desc = clean_html(i_attr.get("description", ""))
                effect = clean_html(i_attr.get("effect", ""))
                cost = str(i_attr.get("cost_impact", 0))
                rec = clean_html(i_attr.get("recommendation", ""))
                icustom = i_attr.get("custom_attributes", [])
                cm1 = next((f.get("value", "") for f in icustom if f.get("term") == "Custom field 1"), "")
                cm2 = next((f.get("value", "") for f in icustom if f.get("term") == "Custom field 2"), "")
                table_data.append([pid, name, branch, region, start, status, title, severity, desc, effect, cost, cm1, cm2, rec, bm, om, sup])
        except Exception as e:
            logger.error(f"Error processing project {project.get('id')}: {e}")

    if table_data:
        pd.DataFrame(table_data, columns=headers).to_csv("project_data.csv", index=False)
        doc = create_word_report(table_data)
        doc.save("project_report.docx")
        logger.info("✅ Report generated and saved.")
    else:
        logger.warning("⚠️ No qualifying data to generate report.")

if __name__ == "__main__":
    main()


2025-06-16 13:53:38,916 - WARNING - Header logo loading failed: [Errno 2] No such file or directory: 'minigroup_logo.png'
2025-06-16 13:53:39,128 - WARNING - Header logo loading failed: [Errno 2] No such file or directory: 'minigroup_logo.png'
/tmp/ipykernel_23415/458835984.py:36: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(value, 'html.parser')
2025-06-16 13:53:39,279 - WARNING - Header logo loading failed: [Errno 2] No such file or directory: 'minigroup_logo.png'
2025-06-16 13:53:39,712 - WARNING - Header logo loading failed: [Errno 2] No such file or directory: 'minigroup_logo.png'
2025-06-16 13:53:39,892 - WARNING - Header logo loading failed: [Errno 2] No such file or directory: 'minigroup_logo.png'
2025-06-16 13:53:39,999 - WARNING - Header logo loading failed: [Errno 2] No such file or directory: 'minigroup_logo.png'
2025-06-16 13:53:40,231 - WA

In [6]:
import sys
print(sys.executable)
print(sys.path[:3], "…")


/home/kitavidouglas/anaconda3/bin/python
['/home/kitavidouglas/Desktop/DataAnalyst', '/home/kitavidouglas/anaconda3/lib/python312.zip', '/home/kitavidouglas/anaconda3/lib/python3.12'] …
